In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import openpyxl
from datetime import datetime
from openpyxl import load_workbook

# Função para extrair informações do site e salvar na planilha
def extrair_informacoes(link, linha):
    requisicao = requests.get(link)
    site = BeautifulSoup(requisicao.text, "html.parser")
    
    nome_produto = site.find(class_='woocommerce-product-details__brand')
    
    preco_elemento = site.find('bdi')
    if preco_elemento:
        preco_produto = preco_elemento.get_text()
        
        # Remover o símbolo "$"
        preco_produto = preco_produto.replace('$', '')
        
        # Verificar se o preço é maior que mil
        if ',' in preco_produto and '.' in preco_produto:
            # Trocar a vírgula por ponto e o ponto por vírgula
            preco_produto = preco_produto.replace(',', '').replace('.', ',', 1)
        elif '.' in preco_produto:
            # Trocar o ponto por vírgula
            preco_produto = preco_produto.replace('.', ',')
        elif ',' in preco_produto:
            # Remover a vírgula dos milhares
            preco_produto = preco_produto.replace(',', '')
            # Trocar o ponto por vírgula
            preco_produto = preco_produto.replace('.', ',')
        
        # Forçar o tipo de dado para número
        preco_produto = float(preco_produto.replace(',', '.'))
    else:
        preco_produto = None

    sku_produto = site.find(class_='sku')

    image_tag = site.find('div', {'data-thumb': True})

    # Verificar se a tag de imagem foi encontrada
    if image_tag:
        image_link = image_tag['data-thumb']
    else:
        image_link = None

    # Encontrando a próxima linha vazia na planilha
    proxima_linha = sheet.max_row + 1
    
    # Salvando as informações na planilha na próxima linha vazia
    sheet[f'A{proxima_linha}'] = nome_produto.text if nome_produto else ''
    sheet[f'B{proxima_linha}'] = preco_produto if preco_produto else ''
    sheet[f'C{proxima_linha}'] = sku_produto.text if sku_produto else ''
    sheet[f'D{proxima_linha}'] = image_link
    sheet[f'F{proxima_linha}'] = 'PRV Audio USA'
    
    # Adicionando a data na próxima linha vazia
    sheet[f'E{proxima_linha}'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # Definir o formato da célula como número
    sheet[f'B{proxima_linha}'].number_format = '#,##0.00'

# Lendo os links da planilha Excel
planilha = pd.read_excel('03.Input_links_PRV_audio.xlsx')

# Carregando a planilha de produtos existente
workbook = openpyxl.load_workbook('02.Output_informacoes_produtos_PRV_audio.xlsx')
sheet = workbook.active

# Verificando se as colunas já existem na planilha
if 'Nome do Produto' not in sheet['A1'].value:
    sheet['A1'] = 'Nome do Produto'
if 'Preço do Produto' not in sheet['B1'].value:
    sheet['B1'] = 'Preço do Produto'
if 'SKU do Produto' not in sheet['C1'].value:
    sheet['C1'] = 'SKU do Produto'
if 'Imagem do Produto' not in sheet['D1'].value:
    sheet['D1'] = 'Imagem do Produto'   
if 'Data da Extração' not in sheet['E1'].value:
    sheet['E1'] = 'Data da Extração'
if sheet['F1'].value is None or 'Origem' not in sheet['F1'].value:
    sheet['F1'] = 'Origem'

# Iterando sobre cada linha da planilha de links
for idx, row in planilha.iterrows():
    link = row['Link']
    extrair_informacoes(link, idx + 2)  # Começa da linha 2
    
# Salvando a planilha
workbook.save('02.Output_informacoes_produtos_PRV_audio.xlsx')

ChunkedEncodingError: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))